In [ ]:
!pip install tensorflow==2.12
!pip install keras==2.12
!pip install music21
!pip install numpy

### Load

In [ ]:
import os
import numpy as np
from music21 import converter, pitch, interval, instrument, note, stream
from matplotlib import pyplot as plt

from xen.visualise import plotPart, plotSparseNoteSequence
from xen.data.songdata import SongDataSet

data_dir = "../../../ml_training/music/mutopia_guitar/"

dataset = SongDataSet()
dataset.loadMidiDir(data_dir)

print(f'Loaded {len(dataset.songs)} Songs') 


### Filter

In [ ]:
dataset.filterTimeSig('4/4')
print(f'{len(dataset.songs)} Songs') 

###  Encode

In [ ]:
from xen.data.codecs import SparseNoteSequenceCodec, FlatNoteSequenceCodec

ticksPerQuarter = 4   # 4 = 16th notes, to allow triplets would need to be 12 = 48 per measure
quartersPerMeasure = 4
measuresPerSequence = 1

codec = FlatNoteSequenceCodec(ticksPerQuarter, quartersPerMeasure, measuresPerSequence, '4/4')
codec.encodeAll(dataset)

print(dataset.sequences.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from xen.models.VariationalAutoencoder import VariationalAutoEncoder

autoencoder = VariationalAutoEncoder(layerDims=[dataset.sequences.shape[1], 256, 32, 4])
autoencoder.compile(optimizer=Adam(learning_rate=0.005))

### Train

In [ ]:


autoencoder.train(dataset.sequences, batchSize = 32, epochs = 500)


In [ ]:


def countMatches(indata, outdata):
    matches = 0
    for i in range(0, len(indata)):
        insequence = indata[i]
        outsequence = outdata[i]
        match = True
        for j in range(len(insequence)):
            if ((insequence[j] >= 0.5 and outsequence[j] < 0.5) or (insequence[j] < 0.5 and outsequence[j] >= 0.5)):
                match = False
                # print(i)
        if (match):
            matches = matches + 1
    return matches
    

output = autoencoder.predict(dataset.sequences)

matches = countMatches(dataset.sequences, output)
print(f'{matches/len(dataset.sequences)*100}% recall')



### Visualise Latent Space

In [ ]:
latentdata = autoencoder.encode(dataset.sequences)
sampling = np.array(latentdata[0])
print(sampling[:,0])

plt.figure(figsize=(20, 4))
plt.scatter(sampling[:,0], sampling[:,1], color='b', marker='.')
plt.show()

plt.figure(figsize=(20, 4))
plt.scatter(sampling[:,2], sampling[:,3], color='b', marker='.')
plt.show()

plt.figure(figsize=(20, 4))
plt.scatter(sampling[:,4], sampling[:,5], color='b', marker='.')
plt.show()

In [ ]:
index = 2

print(output.shape)


plotSparseNoteSequence(codec.decode(dataset.sequences[index:index+1])[0])
plotSparseNoteSequence(codec.decode(output[index:index+1])[0], threshold = 0.5)


print(np.amax(output, axis = 1))

### Save Model

In [6]:
metadata = {
    "timeSignature": codec.timesignature,
    "sequenceLength": ticksPerQuarter*quartersPerMeasure*measuresPerSequence
}

autoencoder.save("../models", "mutopia_guitar_4-4", metadata)